# Project - Autoencoder

The data set for this exercise is from the banking industry. It contains data about the home loans of 2,500 bank clients. Each row represents a single loan. The columns include the characteristics of the client who used a loan. We will build an autoencoder to learn the representation of good loans. That way, we can see if an unknown loan will be a good or bad loan based on its reconstruction error.  

Note: in the data set, we don't have a column that indicates whether a loan is "good" or "bad". So, we can't train a classification model like we did before.

## Description of Variables

The description of variables are provided in "Loan - Data Dictionary.docx"

# Read and Prepare the Data

In [1]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

C:\Users\Rishabh\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Get the data

In [2]:
good = pd.read_csv('good loans.csv')

unknown = pd.read_csv('unknown loans.csv')

In [3]:
# Note that there is no GOOD/BAD classification. Though, we know that
# these loans are good

good.head()

,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,25900,61064.0,94714.0,DebtCon,Office,2.0,0.0,0.0,98.809375,0.0,23.0,34.565944
1,26100,113266.0,182082.0,DebtCon,Sales,18.0,0.0,0.0,304.852469,1.0,31.0,33.193949
2,20900,62615.0,87904.0,DebtCon,Office,5.0,NaN,NaN,177.864849,NaN,15.0,36.831076
3,25300,62540.0,101165.0,DebtCon,ProfExe,0.0,0.0,0.0,195.451331,0.0,25.0,35.200865
4,27700,73148.0,101462.0,DebtCon,ProfExe,10.0,0.0,0.0,264.605389,0.0,33.0,40.475793


In [4]:
good.shape

(1489, 12)

In [5]:
unknown.shape

(1011, 12)

# Data Prep

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

##  Identify the numeric, binary, and categorical columns

In [7]:
# Identify the numerical columns
numeric_columns = good.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = good.select_dtypes('object').columns.to_list()

In [8]:
numeric_columns

['LOAN',
 'MORTDUE',
 'VALUE',
 'YOJ',
 'DEROG',
 'DELINQ',
 'CLAGE',
 'NINQ',
 'CLNO',
 'DEBTINC']

In [9]:
categorical_columns

['REASON', 'JOB']

# Pipeline

In [10]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [11]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [12]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns)],   
        remainder='passthrough')

#passtrough is an optional step.

# Transform: fit_transform() for GOOD

In [13]:
#Fit and transform the train data
good_x = preprocessor.fit_transform(good)

good_x

array([[ 0.6365782 , -0.29793911, -0.12177109, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.6546228 ,  1.02893596,  1.62034212, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.18546313, -0.25851565, -0.25756212, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 4.18234262, -0.04383447,  0.81444926, ...,  0.        ,
         0.        ,  0.        ],
       [-1.30321659, -0.47828045, -0.73681873, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.03208401, -0.46521555, -0.38146895, ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
good_x.shape

# Tranform: transform() for UNKNOWN

In [14]:
# Transform the test data
unknown_x = preprocessor.transform(unknown)

unknown_x

array([[ 2.81095281,  3.755331  ,  3.98956663, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.32079765, -0.54180025, -0.65167516, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.19448543,  0.59006724,  0.48213012, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [-0.79796772,  0.05628738,  0.05541527, ...,  0.        ,
         0.        ,  0.        ],
       [-0.16640662, -0.510536  , -0.45504735, ...,  0.        ,
         0.        ,  0.        ],
       [-0.81601232, -0.92230904, -0.93360606, ...,  0.        ,
         0.        ,  0.        ]])

In [15]:
unknown_x.shape

(1011, 20)

# Build an Autoencoder to learn the representation of GOOD loans

In [16]:
import tensorflow as tf
from tensorflow import keras

In [20]:
model = keras.models.Sequential()

#Encoder
model.add(keras.layers.InputLayer(input_shape=20))
model.add(keras.layers.Dense(18, activation='relu'))
model.add(keras.layers.Dense(17, activation='relu'))

#Decoder
model.add(keras.layers.Dense(18, activation='relu'))
model.add(keras.layers.Dense(20, activation=None))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 18)                378       
                                                                 
 dense_13 (Dense)            (None, 17)                323       
                                                                 
 dense_14 (Dense)            (None, 18)                324       
                                                                 
 dense_15 (Dense)            (None, 20)                380       
                                                                 
Total params: 1405 (5.49 KB)
Trainable params: 1405 (5.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
adam = keras.optimizers.Adam(learning_rate=0.001)


model.compile(loss='mse', optimizer=adam, metrics=['mean_squared_error'])

In [22]:
from tensorflow.keras.callbacks import EarlyStopping

earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

### Check the average MSE on the "good" loans

In [24]:
# Be careful: both input and output are "good_x" while training the autoencoder

model.fit(good_x, good_x, 
          validation_data = (good_x, good_x),
          epochs=100, batch_size=100, callbacks=callback)

Epoch 1/100


15/15 [==============================] - 26s 796ms/step - loss: 0.6475 - mean_squared_error: 0.6475 - val_loss: 0.6148 - val_mean_squared_error: 0.6148
Epoch 2/100
15/15 [==============================] - 10s 744ms/step - loss: 0.5990 - mean_squared_error: 0.5990 - val_loss: 0.5839 - val_mean_squared_error: 0.5839
Epoch 3/100
15/15 [==============================] - 5s 378ms/step - loss: 0.5743 - mean_squared_error: 0.5743 - val_loss: 0.5633 - val_mean_squared_error: 0.5633
Epoch 4/100
15/15 [==============================] - 5s 378ms/step - loss: 0.5544 - mean_squared_error: 0.5544 - val_loss: 0.5428 - val_mean_squared_error: 0.5428
Epoch 5/100
15/15 [==============================] - 6s 398ms/step - loss: 0.5328 - mean_squared_error: 0.5328 - val_loss: 0.5198 - val_mean_squared_error: 0.5198
Epoch 6/100
15/15 [==============================] - 5s 346ms/step - loss: 0.5073 - mean_squared_error: 0.5073 - val_loss: 0.4917 - val_mean_squared_error: 0.4917
Epoch 7/100
15/15 

15/15 [==============================] - 4s 288ms/step - loss: 0.0514 - mean_squared_error: 0.0514 - val_loss: 0.0506 - val_mean_squared_error: 0.0506
Epoch 49/100
15/15 [==============================] - 5s 342ms/step - loss: 0.0504 - mean_squared_error: 0.0504 - val_loss: 0.0497 - val_mean_squared_error: 0.0497
Epoch 50/100
15/15 [==============================] - 5s 380ms/step - loss: 0.0495 - mean_squared_error: 0.0495 - val_loss: 0.0490 - val_mean_squared_error: 0.0490
Epoch 51/100
15/15 [==============================] - 5s 362ms/step - loss: 0.0486 - mean_squared_error: 0.0486 - val_loss: 0.0481 - val_mean_squared_error: 0.0481
Epoch 52/100
15/15 [==============================] - 5s 351ms/step - loss: 0.0480 - mean_squared_error: 0.0480 - val_loss: 0.0472 - val_mean_squared_error: 0.0472
Epoch 53/100
15/15 [==============================] - 5s 354ms/step - loss: 0.0472 - mean_squared_error: 0.0472 - val_loss: 0.0467 - val_mean_squared_error: 0.0467
Epoch 54/100
15/15 [=========

15/15 [==============================] - 5s 360ms/step - loss: 0.0220 - mean_squared_error: 0.0220 - val_loss: 0.0219 - val_mean_squared_error: 0.0219
Epoch 99/100
15/15 [==============================] - 5s 340ms/step - loss: 0.0217 - mean_squared_error: 0.0217 - val_loss: 0.0213 - val_mean_squared_error: 0.0213
Epoch 100/100
15/15 [==============================] - 4s 318ms/step - loss: 0.0212 - mean_squared_error: 0.0212 - val_loss: 0.0209 - val_mean_squared_error: 0.0209


In [25]:
model.evaluate(good_x, good_x)

47/47 [==============================] - 5s 2ms/step - loss: 0.0209 - mean_squared_error: 0.0209


[0.020916134119033813, 0.020916134119033813]

In [26]:
#Multiply by 100 to make sense of the error term:

model.evaluate(good_x, good_x)[0]*100

47/47 [==============================] - 4s 2ms/step - loss: 0.0209 - mean_squared_error: 0.0209


2.0916134119033813

### Check the average MSE on the "unknown" data

In [27]:
model.evaluate(unknown_x, unknown_x)

32/32 [==============================] - 5s 2ms/step - loss: 0.1108 - mean_squared_error: 0.1108


[0.11080650240182877, 0.11080650240182877]

In [28]:
#Multiply by 100 to make sense of the error term:

model.evaluate(unknown_x, unknown_x)[0]*100

32/32 [==============================] - 5s 3ms/step - loss: 0.1108 - mean_squared_error: 0.1108


11.080650240182877

In [29]:
from sklearn.metrics import mean_squared_error

for i in range(0,20):
    prediction = model.predict(unknown_x[i:i+1])
    print((mean_squared_error(unknown_x[i:i+1], prediction))*100)

    
#Error terms are multiplied by 100 to make sense of the numbers

1/1 [==============================] - 11s 11s/step
21.808922863378605
1/1 [==============================] - 8s 8s/step
2.27550331493745
1/1 [==============================] - 9s 9s/step
5.2856192979220475
1/1 [==============================] - 8s 8s/step
0.15795223067802203
1/1 [==============================] - 9s 9s/step
1.7356653885343278
1/1 [==============================] - 4s 4s/step
0.8001369277406158
1/1 [==============================] - 4s 4s/step
2.5318034510241336
1/1 [==============================] - 4s 4s/step
1.4028946757676626
1/1 [==============================] - 4s 4s/step
170.1774907790515
1/1 [==============================] - 4s 4s/step
8.07259260806776
1/1 [==============================] - 4s 4s/step
2.744017007624983
1/1 [==============================] - 4s 4s/step
2.673122170824543
1/1 [==============================] - 4s 4s/step
1.7282940770005055
1/1 [==============================] - 5s 5s/step
0.3800351024394455
1/1 [==============================] -

In [30]:
for i in range(0,20):
    prediction = model.predict(good_x[i:i+1])
    print((mean_squared_error(good_x[i:i+1], prediction))*100)

    
#Error terms are multiplied by 100 to make sense of the numbers

1/1 [==============================] - 4s 4s/step
0.37702512927165727
1/1 [==============================] - 4s 4s/step
6.750162167532078
1/1 [==============================] - 4s 4s/step
0.5420387961024069
1/1 [==============================] - 4s 4s/step
1.5997976117759765
1/1 [==============================] - 4s 4s/step
1.6081127784643185
1/1 [==============================] - 4s 4s/step
0.5174439162171733
1/1 [==============================] - 4s 4s/step
2.6680403921837126
1/1 [==============================] - 4s 4s/step
2.107385270535313
1/1 [==============================] - 4s 4s/step
2.5162582249978716
1/1 [==============================] - 5s 5s/step
0.8472039412225064
1/1 [==============================] - 4s 4s/step
1.2036493729118005
1/1 [==============================] - 4s 4s/step
0.7079651013624941
1/1 [==============================] - 4s 4s/step
0.3270156119894072
1/1 [==============================] - 4s 4s/step
2.0904911970763793
1/1 [==============================

###"unknown" loans look like good loans or not?

the unknown loads do not look like good loads. The reason is the MSE values for the good_x (times 100) is 2.09 and MSE for the unknown dataset has MSE (times 100) that is way higher at 11.08; which reflects that the unkown loans are not good loans.

Also, 20 predictions from good loans and unknown dataset indicates the difference between the corresponding individual MSE values is very large.